In [1]:
import pandas as pd
import re
from unidecode import unidecode
import pickle

In [2]:
def create_province_key(text):
    u_text = unidecode(text) # Unidecode first help remove special characters
    c_text = str(u_text).lower().strip() # Case must be second step
    c_text = re.sub(r'\sProvince$|^Tinh\s|^Thanh\sPho\s|\sCity$', '', c_text, flags=re.IGNORECASE)
    c_text = re.sub(r"\-|\'", ' ', c_text)
    c_text = c_text.replace(' ', '')
    return c_text

In [3]:
def create_district_key(text):
    if not isinstance(text, str):
        return text
    u_text = unidecode(text) # Unidecode first help remove special characters
    c_text = str(u_text).lower().strip() # Case must be second step
    c_text = re.sub(r"\-|\'", ' ', c_text)
    c_text = c_text.replace(' ', '')
    return c_text

In [4]:
def add_province_key(df_province, province_english, province_key):
    if not df_province[df_province.province_english==province_english].shape[0]:
        raise ValueError(f'{province_english} is not exist in province_english')
    elif df_province[(df_province.province_english==province_english) & (df_province.province_key==province_key)].shape[0]:
        raise ValueError(f'{province_key} is exist in province_key')
    
    df_new = df_province.loc[df_province.province_english==province_english].head(1)
    df_new['province_key'] = province_key
    df_province = pd.concat([df_province, df_new])
    return df_province

In [5]:
def add_district_key(df_district, province_english, district_english, district_key):
    if not df_district[df_district.province_english==province_english].shape[0]:
        raise ValueError(f'{province_english} is not exist in province_english')
    elif not df_district[(df_district.province_english==province_english) & (df_district.district_english==district_english)].shape[0]:
        raise ValueError(f'{district_english} is not exist in district_english of {province_english}')
    elif df_district[(df_district.province_english==province_english) & (df_district.district_english==district_english) & (df_district.district_key==district_key)].shape[0]:
        raise ValueError(f'{district_key} is exist in district_key of {province_english}, {district_english}')
    
    df_new = df_district.loc[(df_district.province_english==province_english) & (df_district.district_english==district_english)].head(1)
    df_new['district_key'] = district_key
    df_district = pd.concat([df_district, df_new])
    return df_district

In [6]:
# Pickle
df = pd.read_csv('../data/output/vietnam_administrative_units.csv')

In [7]:
df

,province,district,ward,long_province,long_district,long_ward,short_district,short_ward,province_english,district_english,ward_english,long_province_english,long_district_english,long_ward_english,short_district_english,short_ward_english,district_level,ward_level,district_level_english,ward_level_english
0,Hà Nội,Ba Đình,Phúc Xá,Thành phố Hà Nội,Quận Ba Đình,Phường Phúc Xá,Ba Đình,Phúc Xá,Ha Noi,Ba Dinh,Phuc Xa,Ha Noi City,Ba Dinh District,Phuc Xa Ward,Ba Dinh,Phuc Xa,Quận,Phường,District,Ward
1,Hà Nội,Ba Đình,Trúc Bạch,Thành phố Hà Nội,Quận Ba Đình,Phường Trúc Bạch,Ba Đình,Trúc Bạch,Ha Noi,Ba Dinh,Truc Bach,Ha Noi City,Ba Dinh District,Truc Bach Ward,Ba Dinh,Truc Bach,Quận,Phường,District,Ward
2,Hà Nội,Ba Đình,Vĩnh Phúc,Thành phố Hà Nội,Quận Ba Đình,Phường Vĩnh Phúc,Ba Đình,Vĩnh Phúc,Ha Noi,Ba Dinh,Vinh Phuc,Ha Noi City,Ba Dinh District,Vinh Phuc Ward,Ba Dinh,Vinh Phuc,Quận,Phường,District,Ward
3,Hà Nội,Ba Đình,Cống Vị,Thành phố Hà Nội,Quận Ba Đình,Phường Cống Vị,Ba Đình,Cống Vị,Ha Noi,Ba Dinh,Cong Vi,Ha Noi City,Ba Dinh District,Cong Vi Ward,Ba Dinh,Cong Vi,Quận,Phường,District,Ward
4,Hà Nội,Ba Đình,Liễu Giai,Thành phố Hà Nội,Quận Ba Đình,Phường Liễu Giai,Ba Đình,Liễu Giai,Ha Noi,Ba Dinh,Lieu Giai,Ha Noi City,Ba Dinh District,Lieu Giai Ward,Ba Dinh,Lieu Giai,Quận,Phường,District,Ward
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10542,Cà Mau,Ngọc Hiển,Viên An Đông,Tỉnh Cà Mau,Huyện Ngọc Hiển,Xã Viên An Đông,Ngọc Hiển,Viên An Đông,Ca Mau,Ngoc Hien,Vien An Dong,Ca Mau Province,Ngoc Hien District,Vien An Dong Commune,Ngoc Hien,Vien An Dong,Huyện,Xã,District,Commune
10543,Cà Mau,Ngọc Hiển,Viên An,Tỉnh Cà Mau,Huyện Ngọc Hiển,Xã Viên An,Ngọc Hiển,Viên An,Ca Mau,Ngoc Hien,Vien An,Ca Mau Province,Ngoc Hien District,Vien An Commune,Ngoc Hien,Vien An,Huyện,Xã,District,Commune
10544,Cà Mau,Ngọc Hiển,Rạch Gốc,Tỉnh Cà Mau,Huyện Ngọc Hiển,Thị trấn Rạch Gốc,Ngọc Hiển,Rạch Gốc,Ca Mau,Ngoc Hien,Rach Goc,Ca Mau Province,Ngoc Hien District,Rach Goc Town,Ngoc Hien,Rach Goc,Huyện,Thị trấn,District,Town
10545,Cà Mau,Ngọc Hiển,Tân Ân,Tỉnh Cà Mau,Huyện Ngọc Hiển,Xã Tân Ân,Ngọc Hiển,Tân Ân,Ca Mau,Ngoc Hien,Tan An,Ca Mau Province,Ngoc Hien District,Tan An Commune,Ngoc Hien,Tan An,Huyện,Xã,District,Commune


In [8]:
df['province_key'] = df['province'].apply(create_province_key)
df['district_key'] = df['short_district'].apply(create_district_key)
df['ward_key'] = df['short_ward'].apply(create_district_key)
df['district_level_english'].fillna('', inplace=True)
df['ward_level_english'].fillna('', inplace=True)

In [9]:
district_count = df[['province', 'long_district', 'short_district']].drop_duplicates()[['province', 'short_district']].value_counts().reset_index()
duplicated_districts = district_count[district_count['count'] > 1]['short_district'].tolist()

In [10]:
# Pickle
duplicated_district_keys = df[df.district.isin(duplicated_districts)]['district_key'].unique().tolist()
duplicated_district_province_keys = df[df.district.isin(duplicated_districts)]['province_key'].unique().tolist()

In [11]:
ward_count = df[['long_province', 'long_district', 'short_ward']].value_counts().reset_index()
df_duplicated_wards = ward_count[ward_count['count'] > 1].copy()
df_duplicated_wards.sort_values(by=['long_province', 'long_district', 'short_ward'], inplace=True)

In [12]:
df_duplicated_wards = df[(df.long_province.isin(df_duplicated_wards.long_province)) & (df.long_district.isin(df_duplicated_wards.long_district)) & (df.short_ward.isin(df_duplicated_wards.short_ward))]

In [13]:
df_duplicated_wards

,province,district,ward,long_province,long_district,long_ward,short_district,short_ward,province_english,district_english,...,long_ward_english,short_district_english,short_ward_english,district_level,ward_level,district_level_english,ward_level_english,province_key,district_key,ward_key
176,Hà Nội,Gia Lâm,Thị trấn Yên Viên,Thành phố Hà Nội,Huyện Gia Lâm,Thị trấn Yên Viên,Gia Lâm,Yên Viên,Ha Noi,Gia Lam,...,Yen Vien Town,Gia Lam,Yen Vien,Huyện,Thị trấn,District,Town,hanoi,gialam,yenvien
178,Hà Nội,Gia Lâm,Xã Yên Viên,Thành phố Hà Nội,Huyện Gia Lâm,Xã Yên Viên,Gia Lâm,Yên Viên,Ha Noi,Gia Lam,...,Yen Vien Commune,Gia Lam,Yen Vien,Huyện,Xã,District,Commune,hanoi,gialam,yenvien
1479,Lai Châu,Mường Tè,Thị trấn Mường Tè,Tỉnh Lai Châu,Huyện Mường Tè,Thị trấn Mường Tè,Mường Tè,Mường Tè,Lai Chau,Muong Te,...,Muong Te Town,Muong Te,Muong Te,Huyện,Thị trấn,District,Town,laichau,muongte,muongte
1484,Lai Châu,Mường Tè,Xã Mường Tè,Tỉnh Lai Châu,Huyện Mường Tè,Xã Mường Tè,Mường Tè,Mường Tè,Lai Chau,Muong Te,...,Muong Te Commune,Muong Te,Muong Te,Huyện,Xã,District,Commune,laichau,muongte,muongte
1706,Sơn La,Mai Sơn,Thị trấn Hát Lót,Tỉnh Sơn La,Huyện Mai Sơn,Thị trấn Hát Lót,Mai Sơn,Hát Lót,Son La,Mai Son,...,Hat Lot Town,Mai Son,Hat Lot,Huyện,Thị trấn,District,Town,sonla,maison,hatlot
1716,Sơn La,Mai Sơn,Xã Hát Lót,Tỉnh Sơn La,Huyện Mai Sơn,Xã Hát Lót,Mai Sơn,Hát Lót,Son La,Mai Son,...,Hat Lot Commune,Mai Son,Hat Lot,Huyện,Xã,District,Commune,sonla,maison,hatlot
1882,Yên Bái,Trạm Tấu,Thị trấn Trạm Tấu,Tỉnh Yên Bái,Huyện Trạm Tấu,Thị trấn Trạm Tấu,Trạm Tấu,Trạm Tấu,Yen Bai,Tram Tau,...,Tram Tau Town,Tram Tau,Tram Tau,Huyện,Thị trấn,District,Town,yenbai,tramtau,tramtau
1887,Yên Bái,Trạm Tấu,Xã Trạm Tấu,Tỉnh Yên Bái,Huyện Trạm Tấu,Xã Trạm Tấu,Trạm Tấu,Trạm Tấu,Yen Bai,Tram Tau,...,Tram Tau Commune,Tram Tau,Tram Tau,Huyện,Xã,District,Commune,yenbai,tramtau,tramtau
1918,Yên Bái,Yên Bình,Thị trấn Yên Bình,Tỉnh Yên Bái,Huyện Yên Bình,Thị trấn Yên Bình,Yên Bình,Yên Bình,Yen Bai,Yen Binh,...,Yen Binh Town,Yen Binh,Yen Binh,Huyện,Thị trấn,District,Town,yenbai,yenbinh,yenbinh
1937,Yên Bái,Yên Bình,Xã Yên Bình,Tỉnh Yên Bái,Huyện Yên Bình,Xã Yên Bình,Yên Bình,Yên Bình,Yen Bai,Yen Binh,...,Yen Binh Commune,Yen Binh,Yen Binh,Huyện,Xã,District,Commune,yenbai,yenbinh,yenbinh


In [14]:
duplicated_ward_keys = df_duplicated_wards.ward_key.unique().tolist()
duplicated_ward_district_keys = df_duplicated_wards.district_key.unique().tolist()

In [15]:
# After testing module, I added this code
# 'Huyện Quang Bình, Tỉnh Hà Giang' -> quangbinh
# 'Huyện Phù Yên, Tỉnh Sơn La' -> phuyen
# 'Huyện Văn Giang, Tỉnh Hưng Yên' -> angiang
# 'Huyện Quảng Ninh, Tỉnh Quảng Bình' -> quangninh
# 'Bac Lieu, Hoa Binh District -> hoabinh
province_key_order = {
    'hagiang': 1,
    'quangbinh': 2,
    'sonla': 3,
    'phuyen': 4,
    'hungyen': 5,
    'angiang': 6,
    'quangninh': 7,
    'baclieu': 8,
    'hoabinh': 9
}

df['province_key_order'] = df['province_key'].map(province_key_order)

df.sort_values(by='province_key_order', inplace=True)

In [16]:
df_province = df[[col for col in df.columns if 'province' in col]].drop_duplicates()

province_alias_keys = [
    ('Ho Chi Minh', 'hcm'),
    ('Ha Noi', 'hn') # Cause wrong with "Tinh Nghe An", fixed by adding to the end of province_keys_2
]

for key in province_alias_keys:
    province_english, province_key = key
    df_province = add_province_key(df_province, province_english, province_key)

In [17]:
df_district = df[['province_english'] + [col for col in df.columns if 'district' in col]].drop_duplicates()

district_alias_keys = [
    ('An Giang', 'Chau Thanh', 'hueduc'),
    ('Ba Ria - Vung Tau', 'Phu My', 'tanthanh'),
    ('Bac Kan', 'Ba Be', 'chora'),
    ('Bac Lieu', 'Bac Lieu', 'minhhai'),
    ('Binh Duong', 'Tan Uyen', 'chauthanh'),
    ('Binh Duong', 'Thuan An', 'laithieu'),
    ('Binh Phuoc', 'Phuoc Long', 'phuocbinh'),
    ('Ca Mau', 'Dam Doi', 'ngochien'),
    ('Cao Bang', 'Quang Hoa', 'phuchoa'),
    ('Cao Bang', 'Quang Hoa', 'quanguyen'),
    ('Cao Bang', 'Ha Quang', 'thongnong'),
    ('Cao Bang', 'Trung Khanh', 'tralinh'),
    ('Dien Bien', 'Muong Lay', 'laichau'),
    ('Dien Bien', 'Muong Cha', 'muonglay'),
    ('Dong Nai', 'Vinh Cuu', 'vinhan'),
    ('Dong Thap', 'Lap Vo', 'thanhhung'),
    ('Ha Nam', 'Phu Ly', 'hanam'),
    ('Ha Noi', 'Soc Son', 'daphuc'),
    ('Ha Noi', 'Soc Son', 'kimanh'),
    ('Hai Phong', 'Cat Hai', 'catba'),
    ('Hau Giang', 'Vi Thanh', 'mythanh'),
    ('Hau Giang', 'Nga Bay', 'tanhiep'),
    ('Hoa Binh', 'Hoa Binh', 'kyson'),
    ('Khanh Hoa', 'Dien Khanh', 'khanhxuong'),
    ('Kien Giang', 'Kien Luong', 'hatien'),
    ('Lao Cai', 'Lao Cai', 'camduong'),
    ('Nam Dinh', 'Nam Dinh', 'myloc'),
    ('Ninh Binh', 'Hoa Lu', 'giakhanh'),
    ('Ninh Binh', 'Nho Quan', 'hoanglong'),
    ('Ninh Binh', 'Yen Mo', 'tamdiep'),
    ('Phu Tho', 'Cam Khe', 'songthao'),
    ('Quang Nam', 'Nam Giang', 'giang'),
    ('Quang Ngai', 'Tra Bong', 'taytra'),
    ('Quang Ninh', 'Ha Long', 'honggai'),
    ('Quang Ninh', 'Van Don', 'campha'),
    ('Quang Ninh', 'Mong Cai', 'haininh'),
    ('Quang Ninh', 'Quang Yen', 'yenhung'),
    ('Quang Ninh', 'Ha Long', 'hoanhbo'),
    ('Tay Ninh', 'Hoa Thanh', 'phukhuong'),
    ('Thanh Hoa', 'Dong Son', 'dongthieu'),
    ('Thanh Hoa', 'Yen Dinh', 'thieuyen'),
    ('Thanh Hoa', 'Nghi Son', 'tinhgia'),
    ('Ho Chi Minh', 'Can Gio', 'duyenhai'),
    ('Ho Chi Minh', 'Thu Duc', 'quan2'),
    ('Ho Chi Minh', 'Thu Duc', 'quan9'),
    # ('Ho Chi Minh', 'Thu Duc', 'thuduc(quan)'),
    ('Tra Vinh', 'Cang Long', 'chauthanhdong'),
    ('Vinh Long', 'Long Ho', 'cainhum'),
    ('Vinh Long', 'Long Ho', 'chauthanhtay'),
    ('Ben Tre', 'Mo Cay Nam', 'mocay'),
    ('Binh Thuan', 'Ham Thuan Nam', 'hamthuan'),
    ('Ha Noi', 'Nam Tu Liem', 'tuliem'),
    ('Quang Nam', 'Nam Tra My', 'tramy'),
    ('Tien Giang', 'Go Cong Tay', 'gocong')
]

for key in district_alias_keys:
    province_english, district_english, district_key = key
    df_district = add_district_key(df_district, province_english, district_english, district_key)

In [18]:
df_district

,province_english,district,long_district,short_district,district_english,long_district_english,short_district_english,district_level,district_level_english,district_key
579,Ha Giang,Hà Giang,Thành phố Hà Giang,Hà Giang,Ha Giang,Ha Giang City,Ha Giang,Thành phố,City,hagiang
702,Ha Giang,Hoàng Su Phì,Huyện Hoàng Su Phì,Hoàng Su Phì,Hoang Su Phi,Hoang Su Phi District,Hoang Su Phi,Huyện,District,hoangsuphi
716,Ha Giang,Xín Mần,Huyện Xín Mần,Xín Mần,Xin Man,Xin Man District,Xin Man,Huyện,District,xinman
678,Ha Giang,Vị Xuyên,Huyện Vị Xuyên,Vị Xuyên,Vi Xuyen,Vi Xuyen District,Vi Xuyen,Huyện,District,vixuyen
679,Ha Giang,Bắc Mê,Huyện Bắc Mê,Bắc Mê,Bac Me,Bac Me District,Bac Me,Huyện,District,bacme
...,...,...,...,...,...,...,...,...,...,...
9349,Ben Tre,Mỏ Cày Nam,Huyện Mỏ Cày Nam,Mỏ Cày Nam,Mo Cay Nam,Mo Cay Nam District,Mo Cay Nam,Huyện,District,mocay
7314,Binh Thuan,Hàm Thuận Nam,Huyện Hàm Thuận Nam,Hàm Thuận Nam,Ham Thuan Nam,Ham Thuan Nam District,Ham Thuan Nam,Huyện,District,hamthuan
198,Ha Noi,Nam Từ Liêm,Quận Nam Từ Liêm,Nam Từ Liêm,Nam Tu Liem,Nam Tu Liem District,Nam Tu Liem,Quận,District,tuliem
6550,Quang Nam,Nam Trà My,Huyện Nam Trà My,Nam Trà My,Nam Tra My,Nam Tra My District,Nam Tra My,Huyện,District,tramy


In [19]:
hcm_districts = df_district[df_district['district_key'].str.contains(r'quan\d{1,2}')].copy()
hcm_districts['district_key'] = hcm_districts['district_key'].str.replace('quan', 'district')
df_district = pd.concat([df_district, hcm_districts])

In [20]:
df_ward = df[['province_english', 'district_english'] + [col for col in df.columns if 'ward' in col]].drop_duplicates()

In [21]:
number_wards = df_ward[df_ward['ward_key'].fillna('').str.contains(r'phuong\d{1,2}')].copy()
number_wards['ward_key'] = number_wards['ward_key'].str.replace('phuong', 'ward')
df_ward = pd.concat([df_ward, number_wards])

In [22]:
df_ward

,province_english,district_english,ward,long_ward,short_ward,ward_english,long_ward_english,short_ward_english,ward_level,ward_level_english,ward_key
579,Ha Giang,Ha Giang,Quang Trung,Phường Quang Trung,Quang Trung,Quang Trung,Quang Trung Ward,Quang Trung,Phường,Ward,quangtrung
702,Ha Giang,Hoang Su Phi,Tân Tiến,Xã Tân Tiến,Tân Tiến,Tan Tien,Tan Tien Commune,Tan Tien,Xã,Commune,tantien
703,Ha Giang,Hoang Su Phi,Nàng Đôn,Xã Nàng Đôn,Nàng Đôn,Nang Don,Nang Don Commune,Nang Don,Xã,Commune,nangdon
704,Ha Giang,Hoang Su Phi,Pờ Ly Ngài,Xã Pờ Ly Ngài,Pờ Ly Ngài,Po Ly Ngai,Po Ly Ngai Commune,Po Ly Ngai,Xã,Commune,polyngai
705,Ha Giang,Hoang Su Phi,Sán Xả Hồ,Xã Sán Xả Hồ,Sán Xả Hồ,San Xa Ho,San Xa Ho Commune,San Xa Ho,Xã,Commune,sanxaho
...,...,...,...,...,...,...,...,...,...,...,...
10449,Ca Mau,Ca Mau,Phường 5,Phường 5,Phường 5,Ward 5,Ward 5,Ward 5,Phường,Ward,ward5
10450,Ca Mau,Ca Mau,Phường 2,Phường 2,Phường 2,Ward 2,Ward 2,Ward 2,Phường,Ward,ward2
10451,Ca Mau,Ca Mau,Phường 8,Phường 8,Phường 8,Ward 8,Ward 8,Ward 8,Phường,Ward,ward8
10452,Ca Mau,Ca Mau,Phường 6,Phường 6,Phường 6,Ward 6,Ward 6,Ward 6,Phường,Ward,ward6


In [23]:
province_keys = df_province['province_key'].tolist()

# Pickle
province_keys_1 = []
province_keys_2 = []
district_keys = str(df.district_key.unique().tolist())
ward_keys = str(df.ward_key.unique().tolist())
for province_key in province_keys:
    if province_key not in district_keys:
        province_keys_1.append(province_key)
    else:
        province_keys_2.append(province_key)

In [24]:
# Pickle
province_map = {}
for province_key in df_province.province_key.unique():
    province = df_province[df_province['province_key'] == province_key]
    province_record = province.to_dict(orient='records')[0]
    province_map[province_key] = province_record

In [25]:
# Pickle
district_map = {}
for province_english in df_province.province_english.unique():
    district_keys = {}
    for district_key in df_district[df_district.province_english == province_english]['district_key'].unique():
        district_levels = {}
        for district_level_english in df_district[(df_district.province_english == province_english) & (df_district.district_key == district_key)]['district_level_english'].unique():
            district = df_district[(df_district.province_english == province_english) & (df_district.district_key == district_key) & (df_district.district_level_english == district_level_english)]
            district_record = district[[col for col in district.columns if 'district' in col]].to_dict('records')[0]
            district_levels[district_level_english] = district_record
        district_keys[district_key] = district_levels
    
    district_map[province_english] = district_keys

In [26]:
ward_map = {}
for province_english in df_province.province_english.unique():
    districts = {}
    for district_english in df_district[df_district.province_english==province_english].district_english.unique():
        wards = {}
        for ward_key in df_ward[(df_ward.province_english==province_english) & (df_ward.district_english==district_english)].ward_key.unique():
            ward_levels = {}
            for ward_level_english in df_ward[(df_ward.province_english==province_english) & (df_ward.district_english==district_english) & (df_ward.ward_key==ward_key)].ward_level_english.unique():
                ward = df_ward[(df_ward.province_english==province_english) & (df_ward.district_english==district_english) & (df_ward.ward_key==ward_key) & (df_ward.ward_level_english==ward_level_english)]
                ward_record = ward[[col for col in ward.columns if 'ward' in col]].to_dict('records')[0]
                ward_levels[ward_level_english] = ward_record
            wards[ward_key] = ward_levels
        districts[district_english] = wards
    ward_map[province_english] = districts

In [27]:
with open('../vnau_parser/data/parse.pkl', 'wb') as f:
    pickle.dump((duplicated_district_keys, duplicated_district_province_keys, duplicated_ward_keys, duplicated_ward_district_keys, province_keys_1, province_keys_2, province_map, district_map, ward_map), f)